In [ ]:

# Install necessary packages
# !pip install planet

# Import libraries
import os
import json
import requests
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling
from shapely.geometry import box
from planet import api

# Set your Planet API key
PLANET_API_KEY = 'YOUR_API_KEY'

# Authenticate with Planet API
client = api.ClientV1(api_key=PLANET_API_KEY)

# Function to get imagery from Planet
def get_planet_imagery(geometry, start_date, end_date, cloud_cover=0.5):
    filters = [
        api.filters.geom_filter(geometry),
        api.filters.date_range('acquired', gte=start_date),
        api.filters.date_range('acquired', lte=end_date),
        api.filters.range_filter('cloud_cover', gte=0, lte=cloud_cover)
    ]
    query = api.filters.and_filter(*filters)
    request = api.filters.build_search_request(query, item_types=['PSScene4Band'])
    result = client.quick_search(request)
    return result.items_iter(limit=None)

# Function to tile the imagery
def tile_imagery(image_path, tile_size):
    with rasterio.open(image_path) as src:
        width = src.width
        height = src.height
        for i in range(0, width, tile_size):
            for j in range(0, height, tile_size):
                window = Window(i, j, min(tile_size, width - i), min(tile_size, height - j))
                window_transform = rasterio.windows.transform(window, src.transform)
                window_crop = src.read(window=window, out_shape=(src.count, window.height, window.width))
                yield window_crop, window_transform

# Function to create analysis-ready training data
def create_training_data(tile_generator, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for i, (tile_data, tile_transform) in enumerate(tile_generator):
        # Example: perform preprocessing or analysis here
        # For simplicity, just save the tile as a numpy file
        np.save(os.path.join(output_dir, f'tile_{i}.npy'), tile_data)

# Example usage
# Define your AOI (Area of Interest)
aoi = box(10.0, 20.0, 30.0, 40.0)  # Example bounding box

# Get Planet imagery
imagery = get_planet_imagery(aoi, '2024-01-01', '2024-01-31', cloud_cover=0.5)

# Download and tile imagery
for image in imagery:
    image_url = image['_links']['_self']
    image_id = image['id']
    image_path = f'{image_id}.tif'
    response = requests.get(image_url, stream=True)
    with open(image_path, 'wb') as f:
        f.write(response.content)
    tile_generator = tile_imagery(image_path, tile_size=256)
    create_training_data(tile_generator, output_dir='training_data')

# Cleanup - remove downloaded imagery
os.remove(image_path)
